In [3]:
# Bem vindo ao meu projeto de IC

In [4]:
# Esse projeto engloba a análise e Estudo das bibliotecas OpenCv e MediaPipe para captura de movimentos do corpo.

In [7]:
### Etapa 1
# Instalação das bibliotecas

!pip install opencv-python mediapipe

In [ ]:
# Importando as bibliotecas para uso

import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

In [9]:
### Etapa 2
# Configuração do ambiente com suporte a captura de vídeo

In [ ]:
# Capturando a webcam

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cap.release()

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
# Protótipo de detecção de pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

with mp_pose.Pose() as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        cv2.imshow("Pose Detection", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 